In [1]:
import meshio
import os
import numpy as np
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

dim = 3
float_precision = 'float32'
PATH = "/home/aben-ham/Desktop/work/manapy0.4/tests/domain/primary/3D/"
PATH = os.path.join(PATH, 'mesh', 'cuboid.msh')

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
MeshPartition(PATH, dim=dim, conf=running_conf, periodic=[0, 0, 0])
domain = Domain(dim=dim, conf=running_conf)

(int32[:,:], int32[:,:], int32[:,:])
Reading gmsh file ...
Saving partition files ...
Number of Cells: 1000
Number of Vertices: 1331
Local domain contruction ...
(int32[:,:], float64[:,:], int32, float64[:,:], float64[:])
(int32[:,:], float64[:,:], int32, int32)
(int32[:,:], int32, int32[:,:], int32[:,:])
(int32, int64[:], int32[:,:], int32[:,:], int32)
(int32[:,:], int32, int32, int32[:,:], int32)
(int32[:,:], int32[:,:], int32, int32)
(int32[:,:], int32[:,:], int32[:], float64[:,:], float64[:,:], int32, float64[:,:], float64[:,:], float64[:,:], float64[:], float64[:,:], int32[:])
(int32[:,:], float64[:,:], float64[:,:])
(int32[:,:], int32[:,:], int32[:,:], int32[:], int32[:], int32[:], int32, int32, int32, int32)
(float64[:,:,:], int32[:,:])
(float64[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], float64[:,:], float64[:,:], float64[:,:], float64[:,:], float64[:], float64[:], float64[:], float64[:], float64[:], float64[:], int32[:], float64[:,:])
(int32[:,:], int32[

In [2]:
  d_cells = domain._cells._nodeid
  d_points = domain._nodes._vertex
  d_center = domain._cells._center
  d_volume = domain._cells._volume

In [10]:
d_points[d_cells[0:8]][:, :, 0:3]

array([[[10. ,  5. , 15. ],
        [ 9. ,  5. , 15. ],
        [ 9. ,  5. , 13.5],
        [10. ,  5. , 13.5],
        [10. ,  4.5, 15. ],
        [ 9. ,  4.5, 15. ],
        [ 9. ,  4.5, 13.5],
        [10. ,  4.5, 13.5],
        [ 1. ,  0. ,  0. ]],

       [[10. ,  4.5, 15. ],
        [ 9. ,  4.5, 15. ],
        [ 9. ,  4.5, 13.5],
        [10. ,  4.5, 13.5],
        [10. ,  4. , 15. ],
        [ 9. ,  4. , 15. ],
        [ 9. ,  4. , 13.5],
        [10. ,  4. , 13.5],
        [ 1. ,  0. ,  0. ]],

       [[10. ,  4. , 15. ],
        [ 9. ,  4. , 15. ],
        [ 9. ,  4. , 13.5],
        [10. ,  4. , 13.5],
        [10. ,  3.5, 15. ],
        [ 9. ,  3.5, 15. ],
        [ 9. ,  3.5, 13.5],
        [10. ,  3.5, 13.5],
        [ 1. ,  0. ,  0. ]],

       [[10. ,  3.5, 15. ],
        [ 9. ,  3.5, 15. ],
        [ 9. ,  3.5, 13.5],
        [10. ,  3.5, 13.5],
        [10. ,  3. , 15. ],
        [ 9. ,  3. , 15. ],
        [ 9. ,  3. , 13.5],
        [10. ,  3. , 13.5],
        [ 1. ,

In [3]:
m_mesh = meshio.read(PATH)

In [4]:
m_mesh.points[m_mesh.cells['hexahedron'][0]]

array([[10. ,  5. , 15. ],
       [ 9. ,  5. , 15. ],
       [ 9. ,  5. , 13.5],
       [10. ,  5. , 13.5],
       [10. ,  4.5, 15. ],
       [ 9. ,  4.5, 15. ],
       [ 9. ,  4.5, 13.5],
       [10. ,  4.5, 13.5]])

In [42]:
d_cells[0][:-1]

array([  6,  62, 197, 106,  61, 520, 602, 431], dtype=int32)

In [47]:
c_points = d_points[d_cells[0][:-1]][:, 0:3]
print(c_points)

[[10.   5.  15. ]
 [ 9.   5.  15. ]
 [ 9.   5.  13.5]
 [10.   5.  13.5]
 [10.   4.5 15. ]
 [ 9.   4.5 15. ]
 [ 9.   4.5 13.5]
 [10.   4.5 13.5]]


In [32]:
def b():
  With = 10.0
  Height = 5.0
  Depth = 15.0
  StepX = With / 10.0
  StepY = Height / 10.0
  StepZ = Depth / 10.0

  cmp = 0
  for x in np.arange(With, 0.0, -StepX):
    for z in np.arange(Depth, 0.0, -StepZ):
      for y in np.arange(Height, 0.0, -StepY):
        points = np.array([
          [x, y, z],
          [x - StepX, y, z],
          [x - StepX, y, z - StepZ],
          [x, y, z - StepZ],

          [x, y - StepY, z],
          [x - StepX, y - StepY, z],
          [x - StepX, y - StepY, z - StepZ],
          [x, y - StepY, z - StepZ],
        ], dtype=d_points.dtype)
        m_points= m_mesh.points[m_mesh.cells['hexahedron'][cmp]]
        np.testing.assert_almost_equal(points, m_points)
        cmp = cmp + 1

b()

In [57]:
d_cells[[0, 1]][:, :-1]

array([[  6,  62, 197, 106,  61, 520, 602, 431],
       [ 61, 520, 602, 431,  60, 519, 603, 422]], dtype=int32)

In [60]:
d_points[d_cells[[0, 1]][:, :-1]][:, :, 0:3]

array([[[10. ,  5. , 15. ],
        [ 9. ,  5. , 15. ],
        [ 9. ,  5. , 13.5],
        [10. ,  5. , 13.5],
        [10. ,  4.5, 15. ],
        [ 9. ,  4.5, 15. ],
        [ 9. ,  4.5, 13.5],
        [10. ,  4.5, 13.5]],

       [[10. ,  4.5, 15. ],
        [ 9. ,  4.5, 15. ],
        [ 9. ,  4.5, 13.5],
        [10. ,  4.5, 13.5],
        [10. ,  4. , 15. ],
        [ 9. ,  4. , 15. ],
        [ 9. ,  4. , 13.5],
        [10. ,  4. , 13.5]]])

In [64]:
d_points[d_cells[[0, 100, 200, 999]][:, :-1]][:, :, 0:3]

array([[[10. ,  5. , 15. ],
        [ 9. ,  5. , 15. ],
        [ 9. ,  5. , 13.5],
        [10. ,  5. , 13.5],
        [10. ,  4.5, 15. ],
        [ 9. ,  4.5, 15. ],
        [ 9. ,  4.5, 13.5],
        [10. ,  4.5, 13.5]],

       [[ 9. ,  5. , 15. ],
        [ 8. ,  5. , 15. ],
        [ 8. ,  5. , 13.5],
        [ 9. ,  5. , 13.5],
        [ 9. ,  4.5, 15. ],
        [ 8. ,  4.5, 15. ],
        [ 8. ,  4.5, 13.5],
        [ 9. ,  4.5, 13.5]],

       [[ 8. ,  5. , 15. ],
        [ 7. ,  5. , 15. ],
        [ 7. ,  5. , 13.5],
        [ 8. ,  5. , 13.5],
        [ 8. ,  4.5, 15. ],
        [ 7. ,  4.5, 15. ],
        [ 7. ,  4.5, 13.5],
        [ 8. ,  4.5, 13.5]],

       [[ 1. ,  0.5,  1.5],
        [ 0. ,  0.5,  1.5],
        [ 0. ,  0.5,  0. ],
        [ 1. ,  0.5,  0. ],
        [ 1. ,  0. ,  1.5],
        [ 0. ,  0. ,  1.5],
        [ 0. ,  0. ,  0. ],
        [ 1. ,  0. ,  0. ]]])

In [29]:
m_mesh.points[m_mesh.cells['hexahedron'][[998, 999]]]

array([[[1. , 1. , 1.5],
        [0. , 1. , 1.5],
        [0. , 1. , 0. ],
        [1. , 1. , 0. ],
        [1. , 0.5, 1.5],
        [0. , 0.5, 1.5],
        [0. , 0.5, 0. ],
        [1. , 0.5, 0. ]],

       [[1. , 0.5, 1.5],
        [0. , 0.5, 1.5],
        [0. , 0.5, 0. ],
        [1. , 0.5, 0. ],
        [1. , 0. , 1.5],
        [0. , 0. , 1.5],
        [0. , 0. , 0. ],
        [1. , 0. , 0. ]]])

In [23]:
With = 10.0
Height = 5.0
Depth = 15.0
StepX = With / 10.0
StepY = Height / 10.0
StepZ = Depth / 10.0
for y in np.arange(Depth, 0.0, -StepZ):
  print(y)

15.0
13.5
12.0
10.5
9.0
7.5
6.0
4.5
3.0
1.5
